Uses an autoencoder to approximate the habituation/dishabituation process
An autoencoder is afixed ontop of the pre-trained model 
LiMA stim are run through it

Author: VAYZENB

In [14]:
curr_dir = '/home/vayzenbe/GitHub_Repos/LiMA'

import sys
sys.path.insert(1, f'{curr_dir}')
import os, argparse
from collections import OrderedDict
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision import datasets
import torchvision.models as models
import numpy as np
from LoadFrames import LoadFrames
from statistics import mean
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
exp = ['Exp1']

skel = [['23','31', '26'],['31_0', '31_50']]
SF = ['Skel', 'Bulge']
#modelType = ['AlexNet_SN', 'ResNet_SN', 'AlexNet_IN', 'ResNet_IN', 'CorNet_Z', 'CorNet_S','SayCam']

batch_num = 10
#hab_min = 4 #minimum number of habituation trials to 
#batch_num = 10 #how many frames to use at a time
#exp = ['Exp1']
#skel=[['23']]
#SF = ['Skel']
#modelType = ['ResNet_SN',  'ResNet_IN', 'CorNet_Z', 'CorNet_S','SayCam']

#Transformations for ImageNet
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])
inv_normalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],std=[1/0.229, 1/0.224, 1/0.225])
# specify loss function
criterion = nn.MSELoss()


epochs = 1



In [16]:
def define_decoder():
    decoder = nn.Sequential(nn.Conv2d(3,1024,kernel_size=3, stride=2), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1), nn.AdaptiveAvgPool2d(1),
    nn.ReLU(), nn.ConvTranspose2d(1024, 3, 224))
    decoder = decoder.cuda()
    
    
    
    return decoder

In [17]:
def save_model(model, epoch, optimizer, loss, file_path):

    print('Saving model ...')
    #torch.save(model.state_dict(), f'{weights_dir}/cornet_classify_{cond}_{epoch}.pt')
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, file_path)

In [18]:
def save_recon(out_,sk_,sf_, model, stim):
    
    fig,ax = plt.subplots(1)
    ax.set_aspect('equal')

    # Show the image
    #ax.imshow(im)

    out_ = out_.squeeze(0)
    out_ = inv_normalize(out_)
    out_ = out_.cpu().detach()
    ax.imshow(out_.permute(1, 2, 0))
    plt.axis('off')
    #print(f'Results/AE/recon/{model}_{fig_fig_}.png')
    plt.savefig(f'{curr_dir}/Results/AE/recon/{model}_{stim}_skel.png', bbox_inches='tight', pad_inches = 0, dpi=150)
    
    

In [ ]:

for ee in range(0,len(exp)):
    hab_data = np.empty(((len(skel[ee]) * len(SF) *len(modelType)),11), dtype = object)
    hn = 0


    for sk in range(0,len(skel[ee])):
        for sf in SF:
            torch.cuda.empty_cache() #clear GPU memory
            hab_dataset = LoadFrames(f'skels/Figure_{skel[ee][sk]}_{sf}', transform=transform)
            trainloader = torch.utils.data.DataLoader(hab_dataset, batch_size=batch_num, shuffle=True, num_workers = 2, pin_memory=True)

            early_hab = 0.0
            late_hab = []

            #Reset decoder for every object (i.e., make it like a fresh hab session)
            #Create decoder
            decoder = define_decoder()
            decoder.train()

            #set up optimzer
            optimizer = torch.optim.Adam(decoder.parameters(), lr=0.01)
            for ep in range(0,epochs):
                train_loss = 0.0 
                total_loss =0.0
                n = 0
                for frames in trainloader:
                    frames = frames.cuda()


                    optimizer.zero_grad() #zero out gradients from previous epoch

                    decode_out = decoder(frames) #Run features through decoder

                    loss = criterion(decode_out, frames) #Calculate loss

                    # backward pass: compute gradient of the loss with respect to model parameters
                    loss.backward()
                    # perform a single optimization step (parameter update)
                    optimizer.step()


                    train_loss += (loss.item()*frames.size(0))
                    n = n +1
                    #print(train_loss, loss.item()*frames.size(0), n)

                total_loss = train_loss/n

                if ep < hab_min:
                    early_hab += total_loss #track loss for the first 4 trials
                    print(ep, total_loss)
                elif ep >= hab_min:
                    hab_start = early_hab / hab_min #Determine habituation criterion
                    late_hab.append(total_loss) #add current loss to habituation
                    hab_end = mean(late_hab[(len(late_hab)-4):len(late_hab)]) #calcualte mean of last 4 hab trials

                    print(ep, total_loss, hab_start, hab_end)
                    if hab_end < (hab_start/2) and ep >= int(hab_min *2): #test if habituated
                        break

            hab_data[hn,0] =  'skel'
            hab_data[hn,1] =  skel[ee][sk]
            hab_data[hn,2] =  sf
            hab_data[hn,3] =  ep
            hab_data[hn,4] =  hab_start
            hab_data[hn,5] =  hab_end

            print('Saving model', f'Figure_{skel[ee][sk]}_{sf}', ep, hab_start, hab_end)  
             
            save_model(decoder, ep, optimizer, loss, f'{curr_dir}/Weights/decoder/{exp[ee]}_skel_Figure_{skel[ee][sk]}_{sf}.pt')
            np.save(f'{curr_dir}/Weights/decoder/{exp[ee]}_Summary_Skel.npy', hab_data)
            hn = hn + 1
            del decoder



In [39]:

for en, ee in enumerate(exp):
    
    hab_data = np.load(f'{curr_dir}/Weights/decoder/{ee}_Summary_Skel.npy',allow_pickle=True)
    

#for mm in range(0,1):

    #test_data = np.empty(((len(skel[ee]) * len(SF)),hab_data.shape[1]), dtype = object)
    
    exp_result = []
    #habituation index
    hn = 0
    #load habituated stim
    for sk_hab in skel[en]:
        for sf_hab in SF:
            trial_result = []
            torch.cuda.empty_cache() #clear GPU memory
            
            #Load model
            decoder = define_decoder()

            #Load checkpoint from fully trained decoder
            checkpoint = torch.load(f'{curr_dir}/Weights/decoder/{ee}_skel_Figure_{sk_hab}_{sf_hab}.pt')
            #print(f'Weights/decoder/{exp[ee]}_{hab_data[mm,0]}_Figure_{hab_data[mm,1]}_{hab_data[mm,2]}.pt')
            decoder.load_state_dict(checkpoint['model_state_dict'])
            decoder.eval()
            
            #load data for dishabituation
            for sk_dis in skel[en]:
                for sf_dis in SF:
                    #load stim for dishab object
                    hab_dataset = LoadFrames(f'skels/Figure_{sk_dis}_{sf_dis}', transform=transform)
                    trainloader = torch.utils.data.DataLoader(hab_dataset, batch_size=batch_num, shuffle=True, num_workers = 2, pin_memory=True)
            

                    if sk_hab == sk_dis:
                        skel_cat = "same"
                    else:
                        skel_cat = "diff"

                    if sf_hab == sf_dis:
                        sf_cat = "same"
                    else:
                        sf_cat = "diff"

                    total_loss = []
                    
                    with torch.no_grad():

                        for frames in trainloader:
                            frames = frames.cuda()

                            decode_out = decoder(frames) #Run features through decoder

                            loss = criterion(decode_out, frames) #Calculate loss
                            train_loss += (loss.item()*frames.size(0))
                            n = n +1

                            # backward pass: compute gradient of the loss with respect to model parameters
                            #loss.backward()
                            # perform a single optimization step (parameter update)
                            #optimizer.step()                        

                            #print(train_loss, loss.item()*frames.size(0), n)
                        total_loss = train_loss/n
                        dishab_trial = hab_data[hn,:6].tolist() + [sk_dis, sf_dis, skel_cat, sf_cat, total_loss]
                        print(dishab_trial)
                        trial_result.append(dishab_trial)
                        exp_result.append(dishab_trial)

                    #print(ep, total_loss)

                    '''
                    if skel_cat == 'same' and sf_cat == 'same':
                        save_recon(decode_out, skel_cat, sf_cat, hab_data[hn,0],f'Figure_{hab_data[hn,1]}_{hab_data[hn,2]}')
                    '''

            print(test_data[hn])
            hn = hn +1
            np.savetxt(f'{curr_dir}/Results/AE/{ee}_{hab_data[hn,0]}_Figure_{hab_data[hn,1]}_{hab_data[hn,2]}_Result.csv', np.array(trial_result), delimiter=',', fmt= '%s')
            
            
    
    
    np.savetxt(f'{curr_dir}/Results/AE/{ee}_Skel_Result.csv', np.array(exp_result), delimiter=',', fmt= '%s')
            
    #hn = hn + 1



['skel', '23', 'Skel', 8, 25.91045490155617, 2.8429818898439407, '23', 'Skel', 'same', 'same', 1.8270587104718803]
['skel', '23', 'Skel', 8, 25.91045490155617, 2.8429818898439407, '23', 'Bulge', 'same', 'diff', 1.816341903230006]
['skel', '23', 'Skel', 8, 25.91045490155617, 2.8429818898439407, '31', 'Skel', 'diff', 'same', 1.8086226510532786]
['skel', '23', 'Skel', 8, 25.91045490155617, 2.8429818898439407, '31', 'Bulge', 'diff', 'diff', 1.8004612459076776]
['skel', '23', 'Skel', 8, 25.91045490155617, 2.8429818898439407, '26', 'Skel', 'diff', 'same', 1.7943346631395949]
['skel', '23', 'Skel', 8, 25.91045490155617, 2.8429818898439407, '26', 'Bulge', 'diff', 'diff', 1.7875986303324285]
['skel' '23' 'Skel' 8 25.91045490155617 2.8429818898439407 '23' 'Skel'
 'same' 'same' 1.1635782817999523]
['skel', '23', 'Bulge', 8, 12.981665773938097, 1.62459262025853, '23', 'Skel', 'same', 'diff', 1.8087131548739677]
['skel', '23', 'Bulge', 8, 12.981665773938097, 1.62459262025853, '23', 'Bulge', 'same',

In [38]:
exp_result

[[['skel',
   '23',
   'Skel',
   8,
   25.91045490155617,
   2.8429818898439407,
   '23',
   'Skel',
   'same',
   'same',
   1.7198110552005519],
  ['skel',
   '23',
   'Skel',
   8,
   25.91045490155617,
   2.8429818898439407,
   '23',
   'Bulge',
   'same',
   'diff',
   1.703737374777213],
  ['skel',
   '23',
   'Skel',
   8,
   25.91045490155617,
   2.8429818898439407,
   '31',
   'Skel',
   'diff',
   'same',
   1.692899469778223],
  ['skel',
   '23',
   'Skel',
   8,
   25.91045490155617,
   2.8429818898439407,
   '31',
   'Bulge',
   'diff',
   'diff',
   1.681440148824527],
  ['skel',
   '23',
   'Skel',
   8,
   25.91045490155617,
   2.8429818898439407,
   '26',
   'Skel',
   'diff',
   'same',
   1.6734674600037662],
  ['skel',
   '23',
   'Skel',
   8,
   25.91045490155617,
   2.8429818898439407,
   '26',
   'Bulge',
   'diff',
   'diff',
   1.6645594624181588]],
 [['skel',
   '23',
   'Skel',
   8,
   25.91045490155617,
   2.8429818898439407,
   '23',
   'Skel',
   'same'

In [ ]:
sk = '23'
sf = 'Skel'
model = 'ResNet_SN'
ee = 0

In [ ]:
sk = hab_data[mm,1]
sf = hab_data[mm,2]
model = hab_data[mm,0]
ee = 0

In [ ]:
encoder, in_feat = load_model(model) #load encoder
encoder.cuda()
encoder.eval()
#Create decoder
#it gets reset for every object
decoder = nn.Sequential(nn.ReLU(), nn.ConvTranspose2d(in_feat, 3, 224))
decoder = decoder.cuda()

#Load checkpoint from fully trained decoder
checkpoint = torch.load(f'Weights/decoder/{exp[ee]}_{model}_Figure_{sk}_{sf}.pt')
decoder.load_state_dict(checkpoint)
decoder.eval()
decoder.train()
optimizer = torch.optim.Adam(decoder.parameters(), lr=0.01)

In [ ]:
hab_dataset = LoadFrames(f'Frames/Figure_{sk}_{sf}', transform=transform)
trainloader = torch.utils.data.DataLoader(hab_dataset, batch_size=batch_num, shuffle=True, num_workers = 0, pin_memory=False)

In [ ]:
train_loss = 0.0 
n = 0
for frames in trainloader:
    frames = frames.cuda()


    encode_out = encoder(frames) #Get encoder features

    optimizer.zero_grad() #zero out gradients from previous epoch

    decode_out = decoder(encode_out) #Run features through decoder

    loss = criterion(decode_out, frames) #Calculate loss
    print((loss.item()*frames.size(0)))
    train_loss += (loss.item()*frames.size(0))
    n = n +1


    # backward pass: compute gradient of the loss with respect to model parameters
    #loss.backward()
    # perform a single optimization step (parameter update)
    #optimizer.step()
    optimizer.zero_grad() #zero out gradients from previous epoch



    #print(train_loss, loss.item()*frames.size(0), n)
print(train_loss/n)

#print(ep, total_loss)

In [ ]:
#scaler = transforms.Resize((224, 224))
#normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[0.229, 0.224, 0.225])
#to_tensor = transforms.ToTensor()
def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name).convert("RGB")
    #image = Variable(normalize(to_tensor(scaler(image))).unsqueeze(0))
    image = transform(image).unsqueeze(0)
    
    return image     

In [ ]:
#load sample image
IM = image_loader(f'Frames/Figure_{sk}_{sf}/Figure_{sk}_{sf}_150.jpg')
IM = IM.cuda()
print(IM.shape)

In [ ]:
print(f'Weights/decoder/{exp[ee]}_{model}_Figure_{sk}_{sf}.pt')
print(IM.shape)


In [ ]:

out = encoder(IM)
out = decoder(out)
loss = criterion(out, IM)
print(loss.item()*IM.size(0))
#out = out.squeeze(0)
#print(out.shape)

From here on is image preview

In [ ]:
out = out.squeeze(0)
print(out.shape)
inv_normalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
   std=[1/0.229, 1/0.224, 1/0.225]
)

output = inv_normalize(out)
output = output.cpu().detach()
plt.imshow(output.permute(1, 2, 0))

In [ ]:
IM = IM.squeeze(0)
IM = inv_normalize(IM)
IM = IM.cpu().detach()
plt.imshow(IM.permute(1, 2, 0))

From here on is image preview